## RFM Analysis

### Importing libraries

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import datetime as dt
import warnings
warnings.filterwarnings('ignore')

### Importing Data

In [72]:
df = pd.read_csv('Online_Retail.csv')

### Understanding Data

In [73]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Understanding Data

In [74]:
def summary(df):
    display(df.head())
    print('-'*100)
    display(df.info())
    print('-'*100)
    display(df.describe([0.01,0.25,0.50,0.75,0.99]))
    

In [75]:
summary(df)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495478 entries, 0 to 495477
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    495478 non-null  object 
 1   StockCode    495478 non-null  object 
 2   Description  494024 non-null  object 
 3   Quantity     495478 non-null  int64  
 4   InvoiceDate  495478 non-null  object 
 5   UnitPrice    495478 non-null  float64
 6   CustomerID   361878 non-null  float64
 7   Country      495478 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 30.2+ MB


None

----------------------------------------------------------------------------------------------------


,Quantity,UnitPrice,CustomerID
count,495478.000000,495478.000000,361878.000000
mean,8.605486,4.532422,15547.871368
std,227.588756,99.315438,1594.402590
min,-80995.000000,-11062.060000,12346.000000
1%,-2.000000,0.190000,12748.000000
25%,1.000000,1.250000,14194.000000
50%,3.000000,2.100000,15514.000000
75%,10.000000,4.130000,16931.000000
99%,100.000000,16.950000,18223.000000
max,80995.000000,38970.000000,18287.000000


### Data Preparation

#### Excluding Negative Values in Quantity and UnitPrice

- We see negative values on Quantity and UnitPrice which is impossible. We need to exlude these values.

In [76]:
df = df[(df.Quantity>0) & (df.UnitPrice> 0)]

#### Dropping Returned Items

- Returned items indicated with C so we can drop them by filtering

In [77]:
df = df[~df['StockCode'].str.contains('C')]

#### Dropping Duplicated Values

In [78]:
df = df.drop_duplicates()

In [79]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,473995.000000,473995.000000,345460.000000
mean,9.713537,3.869483,15548.179011
std,163.760954,35.077471,1594.486951
min,1.000000,0.001000,12346.000000
25%,1.000000,1.250000,14191.000000
50%,3.000000,2.100000,15518.000000
75%,10.000000,4.130000,16931.000000
max,80995.000000,13541.330000,18287.000000


#### Missing Values

In [80]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     128535
Country             0
dtype: int64

In [81]:
df.dropna(inplace=True)

####  Creating "Total Price" Column

In [82]:
df['Total_Price'] = df['UnitPrice']*df['Quantity']

In [83]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [84]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


###  Creating RFM Dataframe

In [100]:
now =  dt.datetime(2011,12,10)

In [102]:
rfm = df.groupby('CustomerID').agg({'InvoiceDate' : lambda day : (now - day.max()).days,
                               'InvoiceNo': lambda num : len(num),
                              'Total_Price': lambda price : price.sum()
                             
                             
                             })

col_list = ['Recency','Frequency','Monetary']

In [104]:
rfm.columns = col_list

### Building RFM Score

- Note that we reversed the numbers for **Recency** , because lower Recency is better for us.

In [116]:
rfm["R"] = pd.qcut(rfm["Recency"],5,labels=[5,4,3,2,1])

In [117]:
rfm["F"] = pd.qcut(rfm["Frequency"],5,labels=[1,2,3,4,5])

In [118]:
rfm["M"] = pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])

In [129]:
rfm["RFM_Score"] = rfm["R"].astype(str) +rfm["F"].astype(str) + rfm["M"].astype(str)

-  Let's analyze our customer by RFM Score

In [138]:
rfm[rfm.RFM_Score == '111'].sort_values(by='Monetary',ascending=False)

,Recency,Frequency,Monetary,R,F,M,RFM_Score
CustomerID,,,,,,,
17060.0,266,5,234.24,1,1,1,111
13786.0,367,7,230.40,1,1,1,111
13712.0,246,10,228.84,1,1,1,111
17212.0,253,4,226.85,1,1,1,111
16587.0,232,13,225.15,1,1,1,111
...,...,...,...,...,...,...,...
13120.0,238,1,30.60,1,1,1,111
17102.0,261,1,25.50,1,1,1,111
17763.0,263,1,15.00,1,1,1,111


### Segmentation Customers

- We will use Recency and Frequency to segment our customers.

In [143]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [144]:
rfm['Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Score,Segment
CustomerID,,,,,,,,
12346.0,325,1,77183.60,1,1,5,115,Hibernating
12747.0,2,100,4128.71,5,4,5,545,Champions
12748.0,0,4345,32509.54,5,5,5,555,Champions
12749.0,3,196,4014.18,5,5,5,555,Champions
12820.0,3,59,942.34,5,4,4,544,Champions


In [148]:
rfm.groupby('Segment').mean().sort_values('Monetary')

,Recency,Frequency,Monetary
Segment,,,
About to Sleep,52.219178,15.267123,435.219486
Promising,22.428571,7.183673,445.777347
Hibernating,207.501042,12.985417,537.732126
Need Attention,51.410256,40.523077,815.458821
Potential Loyalists,15.753846,33.876923,879.101826
At Risk,167.796154,55.523077,884.872462
Can't Loose,145.901408,180.098592,2268.055930
Loyal Customers,33.410788,152.181189,2428.635339
New Customers,6.145833,7.062500,4111.827083
